Setup

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

Prepare the data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Build the model

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

Train the model

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 16s 37ms/step - loss: 0.3752 - accuracy: 0.8875 - val_loss: 0.0865 - val_accuracy: 0.9747
Epoch 2/15
422/422 [==============================] - 14s 34ms/step - loss: 0.1184 - accuracy: 0.9647 - val_loss: 0.0570 - val_accuracy: 0.9848
Epoch 3/15
422/422 [==============================] - 14s 34ms/step - loss: 0.0874 - accuracy: 0.9735 - val_loss: 0.0467 - val_accuracy: 0.9880
Epoch 4/15
422/422 [==============================] - 14s 34ms/step - loss: 0.0728 - accuracy: 0.9775 - val_loss: 0.0418 - val_accuracy: 0.9888
Epoch 5/15
422/422 [==============================] - 15s 35ms/step - loss: 0.0648 - accuracy: 0.9799 - val_loss: 0.0403 - val_accuracy: 0.9895
Epoch 6/15
422/422 [==============================] - 15s 35ms/step - loss: 0.0578 - accuracy: 0.9826 - val_loss: 0.0357 - val_accuracy: 0.9900
Epoch 7/15
422/422 [==============================] - 14s 34ms/step - loss: 0.0534 - accuracy: 0.9831 - val_loss: 0.0386 - val_accuracy:

Evaluate the trained model

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02430807054042816
Test accuracy: 0.9914000034332275


In [6]:
import os
from PIL import Image

In [14]:
train_folder = 'sorted_training/train'
test_folder = 'sorted_training/test'

def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))
    img_array = np.array(img)
    img_array = img_array.reshape(1, 28, 28, 1)
    img_array = img_array.astype('float32') / 255
    return img_array


total_images = 0
correct_predictions = 0

for folder_name in os.listdir(test_folder):
    folder_path = os.path.join(test_folder, folder_name)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        img_array = load_and_preprocess_image(image_path)
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)
        total_images += 1
        if int(folder_name) == predicted_class:
            correct_predictions += 1
        print(f"Image: {image_name}, Folder: {folder_name}, Predicted Digit: {predicted_class}")

accuracy_percentage = (correct_predictions / total_images) * 100
print(f"\nTotal images: {total_images}")
print(f"\nAccuracy: {accuracy_percentage:.2f}%")


1/1 [==============================] - 0s 23ms/step
Image: captcha_317.jpg_5_digit4.jpg, Folder: 1, Predicted Digit: 2
1/1 [==============================] - 0s 19ms/step
Image: captcha_629.jpg_4_digit4.jpg, Folder: 1, Predicted Digit: 2
1/1 [==============================] - 0s 18ms/step
Image: captcha_846.jpg_5_digit4.jpg, Folder: 1, Predicted Digit: 2
1/1 [==============================] - 0s 19ms/step
Image: captcha_915.jpg_3_digit4.jpg, Folder: 1, Predicted Digit: 2
1/1 [==============================] - 0s 25ms/step
Image: captcha_94.jpg_5_digit4.jpg, Folder: 1, Predicted Digit: 2
1/1 [==============================] - 0s 27ms/step
Image: captcha_102.jpg_2_digit4.jpg, Folder: 2, Predicted Digit: 2
1/1 [==============================] - 0s 48ms/step
Image: captcha_106.jpg_3_digit4.jpg, Folder: 2, Predicted Digit: 2
1/1 [==============================] - 0s 21ms/step
Image: captcha_108.jpg_3_digit4.jpg, Folder: 2, Predicted Digit: 2
1/1 [==============================] - 0s 25ms/st